<a href="https://colab.research.google.com/github/guruprasadsrinivasmurthy/Guru/blob/master/ULMFIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
from fastai.text import * 


In [14]:
path = untar_data(URLs.IMDB_SAMPLE)
path


PosixPath('/root/.fastai/data/imdb_sample')

In [15]:
!git clone "https://github.com/guruprasadsrinivasmurthy/Guru.git"

fatal: destination path 'Guru' already exists and is not an empty directory.


In [16]:
import pandas as pd
import numpy as np


df1 = pd.read_csv('Guru/Tweets.csv')

df1['negativereason'] = df1['negativereason'].replace(np.nan, "")

df1 ['text'] =  df1 ['text'] +df1['negativereason']

df2 = pd.DataFrame(df1, columns= ['airline_sentiment', 'text'])

df2.to_csv (path/'text1.csv', index = False, header=True)


In [17]:
from fastai.text import * 

data_lm = TextLMDataBunch.from_csv(path,'text1.csv')
#
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path,'text1.csv', vocab=data_lm.train_ds.vocab, bs=32)


In [18]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')


In [19]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)


In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,4.565426,3.854170,0.293341,00:26


In [21]:
learn.save_encoder('ft_enc')


In [22]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')


RNNLearner(data=TextClasDataBunch;

Train: LabelList (11711 items)
x: TextList
xxbos @southwestair xxmaj poor customer service displayed over the weekend during the storm . i was stuck on the tarmac for 2.5 hours with no answer!customer xxmaj service xxmaj issue,xxbos @southwestair had a great xxup la flight with xxmaj clarence and xxmaj xxunk ! xxmaj those 2 guys are a xxunk ! 😜 😂 thanks,xxbos @united if you lost my belongings then xxup be xxunk xxmaj luggage,xxbos @jetblue xxmaj thank you for the free flyfi ! ! xxmaj makes an already great airline even better ! # jetblue # xxmaj boston # xxunk # xxunk,xxbos @americanair xxmaj thank you .
y: CategoryList
negative,positive,negative,positive,positive
Path: /root/.fastai/data/imdb_sample;

Valid: LabelList (2929 items)
x: TextList
xxbos @usairways you are horrendous . xxmaj pull your shit xxunk xxmaj tell,xxbos @usairways you have got to be kidding me ! xxmaj no info on xxunk from xxup dca to xxup bgr and your phone line tells me to call

In [23]:
data_clas.show_batch()


text,target
xxbos @united xxmaj hi have a question re future xxmaj flight xxmaj booking xxmaj problems . xxup xxunk - xxup jac 29 / 9 xxup jac - xxup lax 8 / 10 xxup lax - xxup xxunk 13 / 10 . i 'm * xxup xxunk xxmaj what is checked bag allowance for xxup jac - xxup lax ?,neutral
"xxbos @americanair our ft xxunk xxup xxunk / xxup dfw ( w / cx to xxup lga ) tomorrow xxmaj cancelled xxmaj flighted , 4 hrs so far waiting for call back . xxmaj what should we do ? xxmaj need to get to nyccustomer xxmaj service xxmaj issue",negative
"xxbos @southwestair i had to travel to xxmaj savannah , xxup ga , once again , on an airline i do n't "" xxmaj luv "" . xxmaj when r u going to fly me to xxmaj savannah ? xxmaj pl xxrep 4 e ase xxrep 4 !",neutral
xxbos @united xxmaj outbound 2 bags 70 lbs bags xxmaj returning home 1 bag 50 lbs . xxmaj what is your logic ? xxmaj and xxup why ? xxmaj am i writing to a xxmaj machine ? xxmaj human xxup help xxunk xxmaj service xxmaj issue,negative
xxbos @united xxmaj why do i still not have my bags ? xxmaj they arrived xxup xxunk & & website says they 're still at the airport .. 2 1 / 2 hours xxmaj late xxmaj flightr ? ? ? ! ! # nothappylost xxmaj luggage,negative


In [24]:
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,0.425710,0.220132,0.904746,00:40


In [25]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.309420,0.183810,0.917378,00:48


In [26]:
learn.predict("@VirginAmerica Can you find us a flt out of LAX that is sooner than midnight on Monday? That would be great customer service 😃")


(Category tensor(1), tensor(1), tensor([0.0145, 0.7842, 0.2012]))